In [ ]:
import json
import requests
import shapely

import geopandas as gpd
import numpy as np
import pandas as pd
import ra

from pathlib import Path

In [58]:
# rural_temps = {}
buffer_size = 10_000

merged_path = Path(r"C:\Users\lain\OneDrive - Instituto Tecnologico y de Estudios Superiores de Monterrey\population_grids_data\final\reprojected\merged")
for path in merged_path.glob("*.gpkg"):
    df = gpd.read_file(path)

    xmin, ymin, xmax, ymax = df.total_bounds
    xmin -= buffer_size
    ymin -= buffer_size
    xmax += buffer_size
    ymax += buffer_size

    xmin, ymin, xmax, ymax = gpd.GeoSeries([shapely.box(xmin, ymin, xmax, ymax)], crs=df.crs).to_crs("EPSG:4326").total_bounds
    
    response = requests.post(
        "http://localhost:8000/suhi/raster",
        data=json.dumps(dict(
            xmin=xmin,
            ymin=ymin,
            xmax=xmax,
            ymax=ymax,
            year=2023,
            season="Qall"
        )),
    )
    response_json = response.json()
    break
    # rural_temps[f"M{path.stem[:2]}.{path.stem[5:]}"] = response.json()["value"]

In [60]:
arr = np.reshape(response_json["data"], (response_json["height"], response_json["width"]))
arr

array([[ 8.70283984,  8.89282479,  8.73198796, ...,  0.68348765,
         0.51391588,  0.71643357],
       [ 8.65774097,  8.38069146,  7.90767547, ..., -0.86497035,
         0.01146694,  0.36513707],
       [ 8.87601952,  8.31793281,  7.64002551, ..., -0.51937055,
         0.59822704,  0.78707265],
       ...,
       [ 7.14319732,  5.86305388,  5.44934687, ...,  4.44682262,
         3.96697059,  3.28461353],
       [ 7.17221884,  5.76374141,  5.28506037, ...,  4.72216312,
         4.22142319,  3.43850672],
       [ 6.59875104,  5.14058573,  4.77356001, ...,  5.09434752,
         5.34424276,  4.92534542]], shape=(837, 597))

In [55]:
decoded["transform"]

[0.0007186522272956172,
 0.0,
 -102.4668718722637,
 0.0,
 -0.0007186522272956172,
 22.179763691024633,
 0.0,
 0.0,
 1.0]

In [ ]:
hashes = {}

for path in merged_path.glob("*.gpkg"):
    df = gpd.read_file(path)

    xmin, ymin, xmax, ymax = df.total_bounds
    xmin -= 5000
    ymin -= 5000
    xmax += 5000
    ymax += 5000

    xmin, ymin, xmax, ymax = gpd.GeoSeries([shapely.box(xmin, ymin, xmax, ymax)], crs=df.crs).to_crs("EPSG:4326").total_bounds
    hashes[f"M{path.stem[:2]}.{path.stem[5:]}"] = str(hash((xmin, ymin, xmax, ymax)))[:7]

In [ ]:
df = pd.Series(rural_temps).to_frame()
df.index.name = "zone"
df.columns = ["temp"]
df.to_csv("./rural_temps.csv")